# Import package

In [4]:
pip install Jetson.GPIO

  Running setup.py bdist_wheel for Jetson.GPIO ... done
  Stored in directory: /root/.cache/pip/wheels/b2/43/d4/efa32294f0705bdd848ce2b655cf8409e30004435bd6c37f8c
Successfully built Jetson.GPIO
Note: you may need to restart the kernel to use updated packages.


In [1]:
!which python
!python --version

/usr/bin/python
Python 2.7.17


# Define

## GPIO pin maps

In [3]:
# Enable Pin 32 / PWM0 from terminal 
# sudo busybox devmem 0x700031fc 32 0x45
# sudo busybox devmem 0x6000d504 32 0x2

# Enable Pin 33 / PWM2
# sudo busybox devmem 0x70003248 32 0x46
# sudo busybox devmem 0x6000d100 32 0x00

/bin/bash: busybox: command not found


In [6]:
!apt-get install busybox

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  busybox
0 upgraded, 1 newly installed, 0 to remove and 17 not upgraded.
Need to get 366 kB of archives.
After this operation, 781 kB of additional disk space will be used.
Get:1 http://ports.ubuntu.com/ubuntu-ports bionic-updates/universe arm64 busybox arm64 1:1.27.2-2ubuntu3.3 [366 kB]
Fetched 366 kB in 1s (401 kB/s)  
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package busybox.
(Reading database ... 44680 files and directories currently installed.)
Preparing to unpack .../busybox_1%3a1.27.2-2ubuntu3.3_arm64.deb ...
Unpacking busybox (1:1.27.2-2ubuntu3.3) ...
Setting up busybox (1:1.27.2-2ubuntu3.3) ...


In [1]:
# Enable Pin 32 / PWM0 from terminal 
!busybox devmem 0x700031fc 32 0x45
!busybox devmem 0x6000d504 32 0x2

# Enable Pin 33 / PWM2
!busybox devmem 0x70003248 32 0x46
!busybox devmem 0x6000d100 32 0x00

In [2]:
import time
import Jetson.GPIO as GPIO
import traitlets
from traitlets.config.configurable import Configurable
from traitlets.config.configurable import SingletonConfigurable


GPIO.setwarnings(False)
GPIO.setmode(GPIO.BOARD)

class DCMotor(Configurable):
    value = traitlets.Float()    
    
    def __init__(self, pwm_pin, ctrl_pin1, ctrl_pin2, **kwargs):
        self.pwm_pin = pwm_pin
        self.ctrl_pin1 = ctrl_pin1
        self.ctrl_pin2 = ctrl_pin2
        # Motor calibration
        self.alpha = kwargs.get('alpha', 1.0)
        self.beta = kwargs.get('beta', 0.0)
        # GPIO setup
        GPIO.setup(self.pwm_pin, GPIO.OUT)
        self.pwm = GPIO.PWM(self.pwm_pin, 50) # (channel, frequency)
        self.pwm.start(0)        
        self.speed = 0
        GPIO.setup(self.ctrl_pin1,GPIO.OUT,initial=GPIO.LOW)
        GPIO.setup(self.ctrl_pin2,GPIO.OUT,initial=GPIO.LOW)
    
    @traitlets.observe('value')
    def _observe_value(self, change):
        self._write_value(change['new'])

    def _write_value(self, value):
        """Sets motor value between [-1, 1]"""
        mapped_value = int(100.0 * (self.alpha * value + self.beta))
        speed = min(max(abs(mapped_value), 0), 100)
        self.set_speed(speed)
        if mapped_value > 0:
            self.rotate_forward()
        else:
            self.rotate_backward()
            
    def set_speed(self, normalized_speed):
        # normalized_speed in percentange from 0 - 100
        self.pwm.ChangeDutyCycle(normalized_speed)
        self.speed = normalized_speed
        
    def rotate_forward(self):
        GPIO.output(self.ctrl_pin1, GPIO.LOW)
        GPIO.output(self.ctrl_pin2, GPIO.HIGH)
    
    def rotate_backward(self):
        GPIO.output(self.ctrl_pin1, GPIO.HIGH)
        GPIO.output(self.ctrl_pin2, GPIO.LOW)
    
    def stop(self):
        GPIO.output(self.ctrl_pin1, GPIO.LOW)
        GPIO.output(self.ctrl_pin2, GPIO.LOW)



In [3]:
class JDuck(SingletonConfigurable):    
    def __init__(self, *args, **kwargs):
        self.left_motor = DCMotor(32,36,38, alpha = 1.0)
        self.right_motor = DCMotor(33,35,37, alpha = 1.0)
        self.left_motor.set_speed(50)
        self.right_motor.set_speed(50)
        
    def set_speeds(self, left_speed, right_speed):
        self.left_motor.set_speed(left_speed)
        self.right_motor.set_speed(right_speed)
        
    def move_forward(self):
        self.left_motor.rotate_forward()
        self.right_motor.rotate_forward()

    def move_backward(self):
        self.left_motor.rotate_backward()
        self.right_motor.rotate_backward()

    def turn_left(self):
        self.left_motor.rotate_backward()
        self.right_motor.rotate_forward()

    def turn_right(self):
        self.left_motor.rotate_forward()
        self.right_motor.rotate_backward()

    def stop(self):
        self.left_motor.stop()
        self.right_motor.stop()
    

In [4]:
jduck = JDuck()

In [4]:
jduck.set_speeds(50,50)

In [44]:
jduck.move_forward()

In [45]:
jduck.stop()

In [6]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  # replace with index of your controller

display(controller)

Controller()

In [10]:
left_link = traitlets.dlink((controller.axes[1], 'value'), (jduck.left_motor, 'value'))
right_link = traitlets.dlink((controller.axes[3], 'value'), (jduck.right_motor, 'value'))

In [17]:
left_link.unlink()
right_link.unlink()

ValueError: list.remove(x): x not in list

In [7]:
# create two sliders with range [-1.0, 1.0]
jduck_speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=1.0, step=0.01, orientation='vertical')
# create a horizontal box container to place the sliders next to each other
# display the container in this cell's output
# display(slider_container)
# create buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# display buttons
# middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
# slider_container = widgets.HBox([left_speed_slider, right_speed_slider])
# controls_box = widgets.VBox([forward_button, middle_box, backward_button, slider_container])
middle_box = widgets.VBox([forward_button, stop_button, backward_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.HBox([jduck_speed_slider,left_button, middle_box, right_button])
display(controls_box)

def stop(change):
    jduck.stop()
    
def move_forward(change):
    jduck.move_forward()
    time.sleep(0.5)

def move_backward(change):
    jduck.move_backward()
    time.sleep(0.5)

def turn_left(change):
    jduck.turn_left()
    time.sleep(0.1)
    jduck.stop()

def turn_right(change):
    jduck.turn_right()
    time.sleep(0.1)
    jduck.stop()
# link buttons to actions
left_link = traitlets.dlink((jduck_speed_slider, 'value'), (jduck.left_motor, 'value'))
right_link = traitlets.dlink((jduck_speed_slider, 'value'), (jduck.right_motor, 'value'))
stop_button.on_click(stop)
forward_button.on_click(move_forward)
backward_button.on_click(move_backward)
left_button.on_click(turn_left)
right_button.on_click(turn_right)

In [8]:
left_link = traitlets.dlink((controller.axes[1], 'value'), (left_motor, 'value'))
right_link = traitlets.dlink((controller.axes[3], 'value'), (right_motor, 'value'))